In [6]:
import pandas as pd
import re
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import kurtosis, normaltest, skew

In [7]:
trades_file_path = [
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_3_day_0_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_3_day_1_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_3_day_2_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_4_day_1_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_4_day_2_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/data/trades_round_4_day_3_wn.csv",
]
prices_file_path = [
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-3-island-data-bottle/prices_round_3_day_0.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-3-island-data-bottle/prices_round_3_day_1.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-3-island-data-bottle/prices_round_3_day_2.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-4-island-data-bottle/prices_round_4_day_1.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-4-island-data-bottle/prices_round_4_day_2.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-4-island-data-bottle/prices_round_4_day_3.csv",
]

In [8]:
TRADER = "Vladmir"

In [9]:
for tfp, pfp in zip(trades_file_path, prices_file_path):
    activities_df = pd.read_csv(pfp, sep=";")
    trade_history_df = pd.read_csv(tfp, sep=";")
    round_day_info = tfp.split("/")[-1].split("_")[2:5]
    round_day = " ".join(round_day_info).upper().replace("_", " ")
    print("\n" + "=" * 100)
    print(f"{round_day}\n" + "=" * 100)

    filtered_trades = trade_history_df[
        (trade_history_df["buyer"] == TRADER) | (trade_history_df["seller"] == TRADER)
    ]
    SYMBOLS_TRADED = sorted(list(filtered_trades["symbol"].unique()))

    for PRODUCT in SYMBOLS_TRADED:
        activities_df = pd.read_csv(pfp, sep=";")
        trade_history_df = pd.read_csv(tfp, sep=";")

        activities_df = activities_df
        trade_history_df = trade_history_df
        mid_price_df = activities_df[["timestamp", "product", "mid_price"]]

        # Pivot the DataFrame to normalize it by timestamp with multiple values
        mid_price_df = mid_price_df.pivot_table(
            index="timestamp",
            columns="product",
            values=["mid_price"],
            # aggfunc="mean",
        ).reset_index()

        # Flatten the MultiIndex in columns and rename appropriately
        mid_price_df.columns = [
            f"{col[1].lower()}_{col[0]}" if col[1] else col[0]
            for col in mid_price_df.columns
        ]

        product_mid_price_column = f"{PRODUCT.lower()}_mid_price"
        # assert product_mid_price_column in mid_price_df.columns

        trade_history_df = trade_history_df[
            (
                (trade_history_df["buyer"] == TRADER)
                | (trade_history_df["seller"] == TRADER)
            )
            & (trade_history_df["symbol"] == PRODUCT)
        ].copy()

        # Computing signed quantity
        trade_history_df["signed_quantity"] = trade_history_df.apply(
            lambda row: (
                row["quantity"]
                if row["buyer"] == TRADER
                else -row["quantity"] if row["seller"] == TRADER else 0
            ),
            axis=1,
        )

        # Creating pivot table
        trade_history_pivot_df = trade_history_df.pivot_table(
            index="timestamp", columns="symbol", values="signed_quantity", aggfunc="sum"
        )

        trade_history_pivot_df.columns = [
            f"{col}_trades_made" for col in trade_history_pivot_df.columns
        ]

        products_trades_made_column = f"{PRODUCT}_trades_made"

        # assert products_trades_made_column in trade_history_pivot_df

        normalized_df = mid_price_df[["timestamp", product_mid_price_column]].copy()
        merged_df = pd.merge(
            normalized_df, trade_history_pivot_df, on="timestamp", how="left"
        )

        # Creating the plot with Plotly
        fig = go.Figure()

        # Add mid price trace
        fig.add_trace(
            go.Scatter(
                x=merged_df["timestamp"],
                y=merged_df[product_mid_price_column],
                mode="lines",
                name="Mid Price",
            )
        )

        # Add trade arrows
        trade_col = f"{PRODUCT}_trades_made"
        if trade_col in merged_df.columns:
            for _, row in merged_df.iterrows():
                if row[trade_col] > 0:  # Buy
                    fig.add_trace(
                        go.Scatter(
                            x=[row["timestamp"]],
                            y=[row[product_mid_price_column]],
                            mode="markers",
                            marker_symbol="triangle-up",
                            marker_color="green",
                            marker_size=10,
                            name="Buy",
                        )
                    )
                elif row[trade_col] < 0:  # Sell
                    fig.add_trace(
                        go.Scatter(
                            x=[row["timestamp"]],
                            y=[row[product_mid_price_column]],
                            mode="markers",
                            marker_symbol="triangle-down",
                            marker_color="red",
                            marker_size=10,
                            name="Sell",
                        )
                    )

        fig.update_layout(
            title=f"Time Series of Mid Price and Trades Made for {PRODUCT} by {TRADER}",
            xaxis_title="Timestamp",
            yaxis_title="Price / Trades Made",
            legend_title="Legend",
            font=dict(family="Courier New, monospace", size=12, color="RebeccaPurple"),
        )

        df = trade_history_df
        df_filtered = df[df["symbol"] == PRODUCT]
        # Filter where trader is either buyer or seller
        df_trader = df_filtered[
            (df_filtered["buyer"] == TRADER) | (df_filtered["seller"] == TRADER)
        ]
        df_trader["cum_qty"] = df_trader["signed_quantity"].cumsum()
        df_trader["cash_value"] = (
            df_trader["price"] * -df_trader["signed_quantity"]
        ).cumsum()
        final_cash_value = df_trader.iloc[-1]["cash_value"]
        final_qty = df_trader.iloc[-1]["cum_qty"]
        # liq_val = df_trader.iloc[-1]["price"]
        liq_val = activities_df[activities_df["product"] == PRODUCT].iloc[-1][
            "mid_price"
        ]
        net_pnl = final_cash_value + (final_qty * liq_val)
        print(f"{TRADER}: Net P&L for {PRODUCT} = {net_pnl}")

        fig.show()


3 DAY 0

3 DAY 1

3 DAY 2

4 DAY 1

4 DAY 2

4 DAY 3
